In [1]:
from train_tf import *
from parameters import *
import socket
import tensorflow as tf

In [2]:

par = get_main_experiments(0, train_gpu=1)
par.enc.opt_model_type = OptModelType.OPT_125m
par.enc.news_source = NewsSource.NEWS_SINGLE
par.train.use_tf_models = True
par.train.batch_size = 32

start = time.time()

temp_save_dir = par.get_res_dir()
print(temp_save_dir, flush=True)
already_processed = os.listdir(temp_save_dir)
save_name = f'{par.grid.year_id}.p'
print('save_name', save_name, "already_processed", already_processed)
if save_name in already_processed:
    print(f'Already processed {save_name}', flush=True)
else:
    trainer = PipelineTrainer(par)
    trainer.def_create_the_datasets(filter_func=lambda x: 'mean' in x.split('/')[-1].split('_'))



START WORKING ON  0
comb 1 / 264
/data/gpfs/projects/punim2039/EightK/res/temp/vec_pred/7e4a5c808328461a05da2c8579131aa7acc51513b2e6791a0d376f97dcdf10be/OPT_125m/NEWS_SINGLE/
save_name 2012.p already_processed []


In [3]:
trainer.train_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [4]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def parse_function(feature, label):
    return feature.numpy(), label.numpy()

def extract_features_and_labels_np(dataset):
    # Use parallel processing to map the parse function
    dataset = dataset.map(lambda x, y: tf.py_function(parse_function, [x, y], [tf.float32, tf.int32]),
                          num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Iterate over batches and concatenate results
    features, labels = zip(*list(dataset))
    return np.concatenate(features), np.concatenate(labels)


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
scaler = StandardScaler()

# Split data
X_train, y_train = extract_features_and_labels_np(trainer.train_dataset)
X_test, y_test = extract_features_and_labels_np(trainer.test_dataset)
X_val, y_val = extract_features_and_labels_np(trainer.val_dataset)


In [6]:
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
ridge_coefficients = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1]
best_score = 0
best_coefficient = None

# Tune the ridge penalty coefficient
for C in ridge_coefficients:
    model = LogisticRegression(penalty='elasticnet', l1_ratio=0.5, C=C, solver='saga', max_iter=50, n_jobs=-1, verbose=1)
    model.fit(X_train_scaled, y_train)
    score = accuracy_score(y_val, model.predict(X_val_scaled))
    if score > best_score:
        best_score = score
        best_coefficient = C

    print(f"Coefficient {C}: Validation Accuracy = {score}")

print(f"Best Coefficient: {best_coefficient}")

# Combine the training and validation sets
X_combined = np.concatenate([X_train, X_val])
y_combined = np.concatenate([y_train, y_val])

final_scaler = StandardScaler()
X_combined_scaled = final_scaler.fit_transform(X_combined)
X_test_scaled = final_scaler.transform(X_test)


# Train the final model on the combined dataset
final_model = LogisticRegression(penalty='elasticnet', l1_ratio=0.5, C=best_coefficient, solver='saga', max_iter=50, n_jobs=-1, verbose=1)
final_model.fit(X_combined_scaled, y_combined)

final_model_pred = final_model.predict(X_test_scaled)
final_model_pred_prb = final_model.predict_proba(X_test_scaled)

print(classification_report(y_test, final_model_pred))
print(confusion_matrix(y_test, final_model_pred))


Coefficient 1e-05: Validation Accuracy = 0.5229772779273809


In [7]:
# Create and train the model
model = LogisticRegression(penalty='elasticnet', l1_ratio=0.5, C=1e-3, solver='saga', max_iter=50, n_jobs=-1, verbose=1)
model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = model.predict(X_val_scaled)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.


max_iter reached after 1233 seconds


C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 20.5min finished


              precision    recall  f1-score   support

           0       0.51      0.19      0.27    460751
           1       0.53      0.84      0.65    505138

    accuracy                           0.53    965889
   macro avg       0.52      0.51      0.46    965889
weighted avg       0.52      0.53      0.47    965889

[[ 85426 375325]
 [ 81466 423672]]


In [8]:
# Get the probability estimates for each class
probabilities = final_model.predict_proba(X_test_scaled)

ids = np.concatenate([id_batch.numpy().astype(str) for _, _, id_batch, _, _ in self.test_dataset_with_id],
                             axis=0)
tickers = np.concatenate(
    [ticker_batch.numpy().astype(int) for _, _, _, _, ticker_batch in self.test_dataset_with_id], axis=0)
dates = np.concatenate([date_batch.numpy().astype(str) for _, _, _, date_batch, _ in self.test_dataset_with_id],
                       axis=0)
results_df = pd.DataFrame({
    'id': ids,
    'date': dates,
    'ticker': tickers,
    'y_true': y_test,
    'y_pred': y_pred,
    'y_pred_prb': probabilities[:, 1]
})
results_df['accuracy'] = results_df['y_pred'] == results_df['y_true']
results_df




NameError: name 'final_model' is not defined

In [9]:
filter_func=lambda x: 'mean' in x.split('/')[-1].split('_')
path_with_records = trainer.par.get_training_dir()
end_val = trainer.par.grid.year_id - 1
start_val = end_val - trainer.par.train.T_val + 1
start_train = trainer.par.grid.year_id - trainer.par.train.T_train
end_train = start_val - 1
start_test = trainer.par.grid.year_id
end_test = trainer.par.grid.year_id - 1 + trainer.par.train.testing_window
# tfrecord_files = [os.path.join(path_with_records, f) for f in os.listdir(path_with_records) if '.tfrecord' in f]
# tfrecord_files = list(filter(filter_func, tfrecord_files))
# trainer.train_dataset = trainer.load_dataset('train',tfrecord_files, trainer.par.train.batch_size, start_train, end_train, shuffle=True)
# trainer.val_dataset = trainer.load_dataset('val',tfrecord_files, trainer.par.train.batch_size, start_val, end_val)
# trainer.test_dataset = trainer.load_dataset('test',tfrecord_files, trainer.par.train.batch_size, start_test, end_test)
# trainer.test_dataset_with_id = trainer.load_dataset('test',tfrecord_files, trainer.par.train.batch_size, start_test, end_test, return_id_too=True)
# sample_batch = next(iter(trainer.train_dataset.take(1)))


print(start_test, end_test)
print(start_val, end_val)
print(start_train, end_train)
print("trainer.par.grid.year_id", trainer.par.grid.year_id)
print("trainer.par.train.T_val", trainer.par.train.T_val)
print("trainer.par.train.T_train", trainer.par.train.T_train)

2012 2012
2010 2011
2004 2009
trainer.par.grid.year_id 2012
trainer.par.train.T_val 2
trainer.par.train.T_train 8


In [6]:

reg_value = trainer.par.train.shrinkage_list[0]
reg = tf.keras.regularizers.l1_l2(reg_value, reg_value)

tr_data=trainer.train_dataset
val_data=trainer.val_dataset
reg_to_use=reg

early_stop = tf.keras.callbacks.EarlyStopping(monitor=trainer.par.train.monitor_metric, patience=trainer.par.train.patience, restore_best_weights=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=trainer.par.train.adam_rate)  # Using AMSGrad variant

layers = [
    tf.keras.layers.Input(shape=(trainer.input_dim,))
]
for features, _ in tr_data.take(1):
    print("Shape of features:", features.shape)  # Debugging statement

# if trainer.par.train.norm == Normalisation.ZSCORE:
#     # Create a Normalization layer
#     normalization_layer = tf.keras.layers.Normalization(axis=-1)
#     normalization_layer.adapt(tr_data.map(lambda x, y: x))
#     layers.append(normalization_layer)

layers.append(
    tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(trainer.input_dim,), kernel_regularizer=reg_to_use)
)

model = tf.keras.models.Sequential(layers)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


Shape of features: (32, 768)


KeyboardInterrupt: 

In [22]:
tr_data.map(lambda x, y: x)

<MapDataset element_spec=TensorSpec(shape=(None, None), dtype=tf.float32, name=None)>

In [23]:
for features, _ in tr_data.take(1):
    print("Batch shape:", features.shape)

Batch shape: (32, 768)


In [25]:
trainer.par.train.shrinkage_list


[0.001, 0.01, 0.1, 1, 10]

In [21]:

normalization_layer = tf.keras.layers.Normalization(axis=-1)
normalization_layer.adapt(tr_data.map(lambda x, y: x))

ValueError: in user code:

    File "C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\keras\engine\base_preprocessing_layer.py", line 122, in adapt_step  *
        self._adapt_maybe_build(data)
    File "C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\keras\engine\base_preprocessing_layer.py", line 295, in _adapt_maybe_build  **
        self.build(data_shape)
    File "C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\keras\layers\preprocessing\normalization.py", line 187, in build
        raise ValueError(

    ValueError: All `axis` values to be kept must have known shape. Got axis: (-1,), input shape: [None, None], with unknown axis at index: 1


In [42]:
history = model.fit(tr_data, validation_data=val_data, epochs=trainer.par.train.max_epoch, callbacks=[early_stop])

Epoch 1/10


C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_3/dense_3/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_3/dense_3/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_3/dense_3/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


TypeError: in user code:

    File "C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\keras\engine\training.py", line 997, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\keras\optimizers\optimizer_v2\optimizer_v2.py", line 579, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "C:\Users\nxcle\.conda\envs\spartan\lib\site-packages\keras\o

In [30]:
for i in range(5):
    par = get_main_experiments(i, train_gpu=1)
    print(par.grid.year_id)

START WORKING ON  0
comb 1 / 264
2012
START WORKING ON  1
comb 2 / 264
2013
START WORKING ON  2
comb 3 / 264
2014
START WORKING ON  3
comb 4 / 264
2015
START WORKING ON  4
comb 5 / 264
2016


In [38]:
early_stop

In [17]:
dataset = tf.data.TFRecordDataset(tfrecord_files)

# Parse the dataset using the provided function
dataset = dataset.map(trainer.parse_tfrecord)

# Convert start_year and end_year to TensorFlow constants
const_start_year = tf.constant(start_train, dtype=tf.int32)
const_end_year = tf.constant(end_train, dtype=tf.int32)

# Filter the dataset based on years
dataset = dataset.filter(lambda x: trainer.filter_start_year(x, const_start_year))
dataset = dataset.filter(lambda x: trainer.filter_end_year(x, const_end_year))


# If this particular dataset must be filtered, we apply the filter at the tfrecords level.
if trainer.par.train.apply_filter is not None:
    if 'train' in trainer.par.train.apply_filter:
        dataset = dataset.filter(lambda x: trainer.filter_sample_based_on_par(x))


print(len(dataset))

TypeError: The dataset length is unknown.

In [18]:
sample = next(iter(trainer.train_dataset.map(lambda x, y: x).take(1)))
sample

<tf.Tensor: shape=(32, 768), dtype=float32, numpy=
array([[ 0.06059948,  0.26095   ,  0.7160798 , ...,  0.85200804,
         0.06284972, -0.2488295 ],
       [-0.02092297, -1.4817846 ,  0.31832224, ..., -0.81877476,
        -0.6297853 ,  0.424152  ],
       [ 0.53160125, -2.2838597 ,  0.7216092 , ..., -1.3637179 ,
        -0.06406772,  0.14400324],
       ...,
       [ 1.3553295 ,  0.01270566,  1.6130308 , ..., -0.28950536,
        -0.06183032,  0.13417225],
       [ 1.4286543 ,  0.09823425,  1.6499714 , ..., -0.28477496,
         0.02212846,  0.13438597],
       [ 0.9099802 , -1.1186858 ,  1.5222925 , ..., -0.05351489,
        -0.13695595, -0.01154403]], dtype=float32)>

In [16]:
sample.shape

TensorShape([32, 768])

In [7]:
sample = next(iter(trainer.train_dataset.take(1)))
sample

(<tf.Tensor: shape=(32, 768), dtype=float32, numpy=
 array([[ 0.99465406,  0.5371273 ,  1.49615   , ...,  0.07846408,
         -0.24202517,  0.12293728],
        [ 0.12876956,  0.18122949,  0.16271731, ..., -0.17749994,
         -0.53009   ,  0.6366029 ],
        [-0.601637  , -0.45197624,  0.63373214, ..., -0.2614409 ,
         -0.2900576 ,  0.88151765],
        ...,
        [ 0.71718615, -0.04619605,  1.3755744 , ...,  0.43199763,
         -0.09834842, -0.39803368],
        [ 1.0380542 ,  0.68655384,  1.4865812 , ...,  0.06338909,
         -0.19234091,  0.08200923],
        [-0.31749234, -0.7784312 ,  0.30476886, ..., -0.70451814,
         -0.05718014,  0.83873475]], dtype=float32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 1, 0, 1, 1, 0])>)

In [62]:
nnz = tf.reduce_sum(tf.cast(tf.math.not_equal(dense_batch, 0), tf.float32))
total_elements = tf.size(dense_batch, out_type=tf.dtypes.float32)
print("nnz =", nnz, "total_elements = ", total_elements)
sparsity = 100 * (1 - nnz / total_elements)
print(sparsity)

nnz = tf.Tensor(24576.0, shape=(), dtype=float32) total_elements =  tf.Tensor(24576.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)


In [67]:
nnz = tf.size(sample[0].values, out_type=tf.float32)
total_elements = tf.reduce_prod(sample[0].dense_shape)
print("nnz =", nnz, "total_elements = ", total_elements)


nnz = tf.Tensor(24576.0, shape=(), dtype=float32) total_elements =  tf.Tensor(24576, shape=(), dtype=int64)


In [68]:
trainer.input_dim


768

In [ ]:
    # train to find which penalisaiton to use
trainer.train_to_find_hyperparams()
trainer.train_on_val_and_train_with_best_hyper()
end = time.time()
print('Ran it all in ', np.round((end - start) / 60, 5), 'min', flush=True)
df = trainer.get_prediction_on_test_sample()
df.to_pickle(temp_save_dir + save_name)

print(df, flush=True)
print('saved to', temp_save_dir + save_name, flush=True)
print('We used',trainer.best_hyper_value)

In [47]:
np.random.choice(100, 10, replace=False)

array([89, 14, 77, 57, 74, 23, 41, 47, 98, 21])